# G-Research Crypto - Starter XGB Pipeline
![](https://storage.googleapis.com/kaggle-competitions/kaggle/30894/logos/header.png)


### Just a simple pipeline going from zero to a valid submission




# Import and load dfs

References: [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import gresearch_crypto
import xgboost as xgb
import traceback
import datetime
import matplotlib.pyplot as plt
from scipy.stats import pearsonr


TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'
DEVICE = 'GPU'

### Reduce memory usage

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
DEVICE = 'GPU'

df = pd.read_csv(TRAIN_CSV)
df = df.replace([np.inf, -np.inf], np.nan)

#df_train.fillna(-999, inplace=True)
# df = df[df['Target'].notna()]
# df.interpolate(method='linear', inplace=True)
df = df.dropna(how="any")

# sorting data into groups of days
df['date'] = pd.to_datetime(df['timestamp'], unit = 's')
df = df.sort_values('date')
groups = pd.factorize(df['date'].dt.day.astype(str) + '_' + df['date'].dt.month.astype(str) + '_' + df['date'].dt.year.astype(str))[0]

# reduce memory usage
df.drop(columns = 'date', inplace = True)
target = df['Target'].copy()
df.drop(columns = 'Target', inplace = True)
df = reduce_mem_usage(df)
df['Target'] = target
df['groups'] = groups

# getting rid of data overlap
df_train = df[df['timestamp'] < 1623542400]
df_train.info()

In [ ]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
df_asset_details

## Utility functions to train a model for one asset

In [ ]:

from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=False, interpolate = False):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    elif interpolate:
        agg.interpolate(method='linear', inplace=True)
    else:
        agg.fillna(-999,inplace=True)
    return agg
 

### Functions to get shadows

In [ ]:
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']


### Simple Moving Average (SMA)

In [ ]:
def SMA(df):
    df['SMA_5'] = df.groupby('Asset_ID')['Close'].transform(lambda x: x.rolling(window = 5).mean())
    df['SMA_15'] = df.groupby('Asset_ID')['Close'].transform(lambda x: x.rolling(window = 15).mean())
    df['SMA_ratio'] = df['SMA_15'] / df['SMA_5']
    return df

### Wilder’s Smoothing

In [ ]:
def Wilder(data,period):
    return data['Close'].ewm(alpha=1.0 / period,adjust=False,).mean()

## One stop get_features function

In [ ]:

# A utility function to build features from the original df
# It works for rows to, so we can reutilize it.
def get_features(df, lag = 1, shuffle = False, less_features = False, diff = True, log= True, sma = True, wilder = True):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    
    df_feat['minute_log_ret'] = np.log(df_feat.Close/df_feat.Open).fillna(-999)
    # params
    if less_features:
        df_feat = df_feat[['Close', 'Volume','Count','minute_log_ret']]
    if sma:
        df_feat[['SMA_5','SMA_15', 'SMA_ratio']] = SMA(df)[['SMA_5','SMA_15', 'SMA_ratio']]
    if wilder:
        df_feat_wilder = Wilder(df, 15)
    if log:
        for column in df_feat.columns:
            if column in ['Open', 'High', 'Low', 'Close', 'VWAP']:
                df_feat[column] = np.log(df_feat[column])
    if diff:
        for column in df_feat.columns:
            if column in ['Open', 'High', 'Low', 'Close', 'VWAP']:
                df_feat[f'diff_{column}'] = df_feat[column].diff().fillna(-999)
    # add lagged observations
    if lag > 0:
        df_feat = series_to_supervised(df_feat, n_in = lag)
    df_feat['Upper_Shadow'] = upper_shadow(df)
    df_feat['Lower_Shadow'] = lower_shadow(df)
    if shuffle is True:
        df_feat = df_feat.sample(frac=1)
    
    df_feat.replace([np.inf, -np.inf], np.nan)
    df_feat.fillna(-999,inplace=True)
    return df_feat

In [ ]:
def calculate_target(data: pd.DataFrame,  price_column: str, asset_details = df_asset_details):
    ids = list(asset_details.Asset_ID)
    asset_names = list(asset_details.Asset_Name)
    weights = np.array(list(asset_details.Weight))

    times = data['timestamp'].agg(['min', 'max']).to_dict()
    all_timestamps = np.arange(times['min'], times['max'] + 60, 60)
    targets = pd.DataFrame(index=all_timestamps)

    for i, id in enumerate(ids):
        asset = data[data.Asset_ID == id].set_index(keys='timestamp')
        price = pd.Series(index=all_timestamps, data=asset[price_column])
        targets[asset_names[i]] = np.log(
            price.shift(periods=-16) /
            price.shift(periods=-1)
        )
    
    targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)
    
    m = targets['m']

    num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
    denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
    beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

    targets = targets - (beta * m.values).T
    targets.drop('m', axis=1, inplace=True)
    
    return targets

In [ ]:
def get_15_min_log_return(df: pd.DataFrame,  price_column: str, asset_details = df_asset_details):
    data = df.copy()
    ids = list(asset_details.Asset_ID)
    asset_names = list(asset_details.Asset_Name)
    weights = np.array(list(asset_details.Weight))
    
    times = data['timestamp'].agg(['min', 'max']).to_dict()
    all_timestamps = np.arange(times['min'], times['max'] + 60, 60)
#     targets = pd.DataFrame(index = all_timestamps)
    

    for i, id in enumerate(ids):
        asset = data[data.Asset_ID == id].set_index(keys='timestamp')
        price = pd.Series(data=asset[price_column])
        targets = np.log(
            price.shift(periods=-16) /
            price.shift(periods=-1)
        )
#         display(targets)
#         display( targets[asset_names[i]].head(20))
#         display(data[data.Asset_ID == id].head(10))
        data.loc[data.Asset_ID == id, '15_log_return'] = targets.fillna(-999).values
    return data

In [ ]:
# df = df_train[df_train["Asset_ID"] == 10]
# df.isna().sum()

In [ ]:
# df = df_train[df_train["Asset_ID"] == 0]
# #impute y
# df['Target'] = df.Target.interpolate(method='slinear')
# df.isna().sum()

In [ ]:
# # Check the model interface
# x = get_features(df_train.iloc[1])
# #y_pred = models[0].predict([x])
# #y_pred[0]
# y_pred = models[0].predict(pd.DataFrame([x]))
# y_pred[0]

# Cross Validation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False,
                 train_gap = 0
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose
        self.train_gap = train_gap

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        train_gap = self.train_gap
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size - train_gap *n_splits,
                                  n_groups, group_test_size + train_gap)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# this is code slightly modified from the sklearn docs here:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
# fig, ax = plt.subplots()

# feature_names = [i for i in df_train.columns if i not in ['Target', 'date', 'timestamp', 'VWAP', 'Asset_ID', 'groups']]
# cv = PurgedGroupTimeSeriesSplit(
#     n_splits=5,
#     max_train_group_size=200,
#     group_gap=10,
#     max_test_group_size=80,
#     train_gap = 100
# )

# plot_cv_indices(
#     cv,    
#     df_train.loc[df_train["Asset_ID"] == 1][feature_names].values,
#     df_train.loc[df_train["Asset_ID"] == 1]['Target'].values > np.nanmean(df_train.loc[df_train["Asset_ID"] == 1]['Target'].values),
#     df_train.loc[df_train["Asset_ID"] == 1]['groups'].values,
#     ax,
#     5,
#     lw=20
# );

In [ ]:
import optuna
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn import set_config

# get_features_params = {'lag':1,'less_features': True, 'shuffle':False, 'diff':True}
feature_names = [i for i in df_train.columns if i not in ['Target', 'date', 'timestamp' 'Asset_ID', 'groups']]
# Testing with Asset_ID == 0 for now
asset_df = get_15_min_log_return(df_train[df_train['Asset_ID'] == 1], 'Close')
# y_labels = asset_df['Target'].values
y_labels = asset_df['15_log_return'].values
# X_train = asset_df[feature_names].values
# X_train = get_features(asset_df[feature_names], **get_features_params).values
groups = asset_df['groups'].values

cv = PurgedGroupTimeSeriesSplit(
    n_splits=5,
    max_train_group_size=200,
    group_gap=20,
    max_test_group_size=80,
    train_gap = 100
)

def objective(trial, cv=cv, cv_fold_func=np.average):
    
    # Optuna suggest params for feature engineering
    feature_params = {
        'lag':trial.suggest_int('lag', 0, 10),
        'less_features': trial.suggest_int('less_features',0, 1), 
        'shuffle': trial.suggest_int('shuffle',0, 1), 
        'diff': trial.suggest_int('diff',0, 1),
        'log': trial.suggest_int('log',0, 1),
        'sma': trial.suggest_int('sma',0, 1),
        'wilder': trial.suggest_int('wilder',0, 1),
        }
    X_train = get_features(asset_df[feature_names], **feature_params).values
    # Optuna suggest params for model
    params = {
        'n_estimators': 550,
        'reg_lambda': trial.suggest_int('reg_lambda', 0, 5),
        "min_child_weight" : trial.suggest_int('min_child_weight', 1, 5),
        'max_depth': trial.suggest_int('max_depth', 6, 15),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.005, 0.02),
        'subsample': trial.suggest_uniform('subsample', 0.50, 0.90),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.40, 0.90),
#         'gamma': trial.suggest_uniform('gamma', 0, 0.5),
        'missing': -999,        
        }
    
    if DEVICE == 'GPU': params['tree_method'] = 'gpu_hist'  
    # setup the pipeline
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    scaler = StandardScaler()
    clf = xgb.XGBRegressor(**params)

    pipe = Pipeline(steps=[
#         ('imputer', imp_mean),
        ('scaler', scaler),
        ('xgb', clf)
    ])


    # fit for all folds and return composite MAE score
    maes = []
    correlations = []
    for i, (train_idx, valid_idx) in enumerate(cv.split(
        X_train,
        y_labels,
        groups=groups)):
        
        train_data = X_train[train_idx, :], y_labels[train_idx]
        valid_data = X_train[valid_idx, :], y_labels[valid_idx]
        
#         display(X_train[valid_idx, :])
        pipe.fit(X_train[train_idx, :], y_labels[train_idx])
        preds = pipe.predict(X_train[valid_idx, :])
        mae = mean_absolute_error(y_labels[valid_idx], preds)
        correlation = pearsonr(y_labels[valid_idx], preds)[0]
#         print(preds)
        maes.append(mae)
        correlations.append(correlation)
    
    print(f'Trial done: mae values on folds: {maes}, correlation: {correlations}')
#     return -1.0 * cv_fold_func(maes)
    return cv_fold_func(correlations)


In [ ]:
%%time

FIT_XGB = False

n_trials = 50

if FIT_XGB:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    best_params = trial.params        
else: best_params = {}

### Best params recorded:

In [ ]:
# best params so far for asset ID = 1, trained on 15_log_return, with more feature engineering
    # Trial 49 finished with value: 0.0317581095907626 and parameters: {'lag': 7, 'less_features': 1, 'shuffle': 1, 'diff': 0, 'log': 0, 'sma': 1, 'wilder': 1, 'reg_lambda': 5, 'min_child_weight': 4, 'max_depth': 15, 'learning_rate': 0.009570687770657272, 'subsample': 0.6285779767805164, 'colsample_bytree': 0.7992539958305589}. Best is trial 22 with value: 0.04930682588394672.
    # Trial done: mae values on folds: [0.0048964624, 0.003702112, 0.003005475, 0.0027706742, 0.0050811954], correlation: [0.011284554536045564, 0.021701535868510636, 0.019523880588945712, 0.040610525761315026, 0.06567005119899602]
    # Number of finished trials: 50
    # Best trial:
    #   Value: 0.04930682588394672
    #   Params: 
    #     lag: 9
    #     less_features: 1
    #     shuffle: 1
    #     diff: 0
    #     log: 0
    #     sma: 0
    #     wilder: 1
    #     reg_lambda: 5
    #     min_child_weight: 1
    #     max_depth: 12
    #     learning_rate: 0.006764463842691914
    #     subsample: 0.7021426413202513
    #     colsample_bytree: 0.7429624696167025
    # CPU times: user 49min 43s, sys: 58.7 s, total: 50min 42s
    # Wall time: 46min 26s
# best params so far for asset ID = 1, trained on 15_log_return
# Trial done: mae values on folds: [0.026105188, 0.026105512, 0.025961224, 0.026012246, 0.026011521], correlation: [0.030052164650627218, 0.029476802869421285, 0.009409405617049048, 0.07442854987788829, 0.057961765589653165]
# Number of finished trials: 200
# Best trial:
#   Value: 0.04418723403026871
#   Params: 
#     lag: 1
#     less_features: 0
#     shuffle: 0
#     diff: 1
#     log: 0
#     n_estimators: 584
#     reg_lambda: 5
#     min_child_weight: 3
#     max_depth: 12
#     learning_rate: 0.005899619552160016
#     subsample: 0.6830598708006316
#     colsample_bytree: 0.8521852685777859
# CPU times: user 3h 35min 51s, sys: 2min 28s, total: 3h 38min 19s
# Wall time: 3h 18min 37s
# best params so far for asset ID = 1
#     Best trial:
#       Value: 0.031607513275039106
#       Params: 
#         lag: 5
#         less_features: 1
#         shuffle: 1
#         diff: 1
#         log: 0
#         n_estimators: 503
#         reg_lambda: 3
#         min_child_weight: 5
#         max_depth: 6
#         learning_rate: 0.010950705188981748
#         subsample: 0.6407943197975113
#         colsample_bytree: 0.4936947696220707
# best params so far for asset ID = 0
#     Trial done: mae values on folds: [0.00884721001153564, 0.008764936314881898, 0.008990281840705116, 0.009167435202743312], correlation: [0.011463596891310724, 0.03417865196140347, 0.007146227215745203, 0.038132198250439626]
#     Number of finished trials: 200
#     Best trial:
#       Value: 0.02448920005394209
#       Params: 
#         lag: 2
#         less_features: 0
#         shuffle: 0
#         diff: 1
#         log: 1
#         n_estimators: 531
#         reg_lambda: 0
#         min_child_weight: 1
#         max_depth: 6
#         learning_rate: 0.011160780508297672
#         subsample: 0.7614506725442451
#         colsample_bytree: 0.8630802285809316
#     CPU times: user 2h 49min, sys: 3min 15s, total: 2h 52min 16s
#     Wall time: 2h 32min 45s

In [ ]:
# maes = []
# correlations = []
# for i, (train_idx, valid_idx) in enumerate(cv.split(
#     X_train,
#     y_labels,
#     groups=groups)):

#     train_data = X_train[train_idx, :], y_labels[train_idx]
#     valid_data = X_train[valid_idx, :], y_labels[valid_idx]

# #         display(X_train[valid_idx, :])
#     pipe = XGB_pipeline()
#     pipe.fit(X_train[train_idx, :], y_labels[train_idx])
#     preds = pipe.predict(X_train[valid_idx, :])
#     mae = mean_absolute_error(y_labels[valid_idx], preds)
#     correlation = pearsonr(y_labels[valid_idx], preds)[0]
# #         print(preds)
#     maes.append(mae)
#     correlations.append(correlation)
# correlations

# Building the model

In [ ]:
# Build pipeline here, imputer might not be necessary as all nans were dropped in the beginning.
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn import set_config
set_config(display='diagram') 
def XGB_pipeline():
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    scaler = StandardScaler()
#     scaler = MinMaxScaler()

    #parameters from optuna optimization
    XGB_model = xgb.XGBRegressor(
        n_estimators=531,
#         learning_rate=0.05,
#         max_depth=12,
#         subsample=0.9,
#         colsample_bytree=0.7,
        #colsample_bylevel=0.75,
        missing=-999,
        min_child_weight= 3,
        reg_lambda= 5,
        max_depth= 12,
        learning_rate = 0.006899619552160016,
        subsample = 0.6830598708006316,
        colsample_bytree = 0.8521852685777859,
#         gamma = 9,
#         reg_lambda = 5,
        random_state=1111,
        tree_method='gpu_hist'  
        )

    pipe = Pipeline(steps=[
        ('imputer', imp_mean),
        ('scaler', scaler),
        ('linear', XGB_model)
    ])
    
    return pipe

# Training

In [ ]:
def get_Xy_and_model_for_asset(df_train, asset_id, 
                               lag = 1, 
                               shuffle = False, 
                               less_features = False, 
                               diff = True, 
                               log = True, 
                               train_on_log_return = False, 
                               sma = True, 
                               wilder = True
                                  ):
    df = df_train[df_train["Asset_ID"] == asset_id]
    #impute y, not working
#     df['Target'] = df.Target.interpolate(method='slinear')
    # TODO: Try different features here!
    X = get_features(df, lag = lag, 
                     shuffle = shuffle, 
                     less_features = less_features, 
                     diff = diff, 
                     log = log,
                     sma = sma, 
                     wilder = wilder).values
    if train_on_log_return:
        y = get_15_min_log_return(df, 'Close')['15_log_return'].values
    else:
        y = df['Target'].values
    #df_proc = df_proc.dropna(how="any")
    
    # TODO: Try different models here!
    #model = LGBMRegressor(random_state=1111, n_estimators=1200)
    #model.fit(X, y)
    #return X, y, model
    
    model = XGB_pipeline()
    
    model.fit(X, y)
    return model

## Loop over all assets

In [ ]:
# asset_df = df_train[df_train['Asset_ID'] == 1]
# X_train = asset_df
# y_labels = asset_df['Target'].values
# cv = PurgedGroupTimeSeriesSplit(
#     n_splits=5,
#     max_train_group_size=200,
#     group_gap=10,
#     max_test_group_size=80,
#     train_gap = 100
# )
# for i, (train_idx, valid_idx) in enumerate(cv.split(
#     asset_df,
#     y_labels,
#     groups=groups)):

#     train_data = asset_df.iloc[train_idx, :]
#     valid_data = asset_df.iloc[valid_idx, :]
    
#     break

In [ ]:
# training models, change params with the 'params' variable.
params = {'lag':9,'less_features': 1, 'shuffle':1, 'diff':0, 'log':0, 'sma': 0, 'wilder':1}

models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    try:
        model = get_Xy_and_model_for_asset(df_train, asset_id, train_on_log_return = 1, **params)  
        models[asset_id] = model
    except KeyboardInterrupt:
        break
    except: 
        traceback.print_exc()
        models[asset_id] = None 

# Testing

In [ ]:
# testing the model, prints out correlation every 1000 iterations.
from tqdm import tqdm

trained_on_15_log_return = False

params['shuffle'] = False
df_test = df[df['Asset_ID'] == 1].copy()
# y_test = df_test[df_test['timestamp'] > 1623542400].Target
X_test = df_test[df_test['timestamp'] > 1623542400].iloc[:-1]
if trained_on_15_log_return:
    # 15_log_return as target
    y_test = get_15_min_log_return(df_test[df_test['timestamp'] > 1623542400], 'Close')['15_log_return'].values
else:
    y_test = df_test[df_test['timestamp'] > 1623542400].Target
all_time_stamps = df_test['timestamp'].drop_duplicates()

## to replicate cross validation results
# df_test = valid_data
# y_test = df_test.Target
# X_test = df_test.iloc[:-1]


dict_tmp = {}
correlations = []
pred = []
counter = 1
wallet = 1400 # 100 per asset, for profit testing
long = {}
buy_price = {}
# m = 0 # for calculating target


for j , row in tqdm(X_test.iterrows()):
    counter += 1
    if models[row['Asset_ID']] is not None:
        try:
            model = models[row['Asset_ID']]
            # putting test data in the correct format, not optimized
            if row['Asset_ID'] not in dict_tmp:
                dict_tmp[row['Asset_ID']] = pd.DataFrame()
            dict_tmp[row['Asset_ID']] = dict_tmp[row['Asset_ID']].append(row)
            if len(dict_tmp[row['Asset_ID']]) > 10:
                dict_tmp[row['Asset_ID']] = dict_tmp[row['Asset_ID']].tail(10)
            x_test = get_features(dict_tmp[row['Asset_ID']],**params).tail(1)
            if len(x_test) < 1 :
                pred.append(0)
                continue
#             y_pred = model.predict(x_test)[0]
            y_pred = model.predict(x_test.values)[0] # convert to nparray
            # testing profit
            if row['Asset_ID'] not in long:
                long[row['Asset_ID']] = 0
            if y_pred > 0 and long[row['Asset_ID']] == 0:
                #buy
                long[row['Asset_ID']] = 1
                buy_price[row['Asset_ID']] = dict_tmp[row['Asset_ID']].tail(1).Close.values[0]
            if y_pred < 0 and long[row['Asset_ID']] ==1:
                #sell
                long[row['Asset_ID']] = 0
                sell_price = dict_tmp[row['Asset_ID']].tail(1).Close.values[0]
                wallet += 100 * sell_price/buy_price[row['Asset_ID']] - 100
                
#             if trained_on_15_log_return:
#                 targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)
#                 m = targets['m']
#                 num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
#                 denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
#                 beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)
#                 targets = targets - (beta * m.values).T
            pred.append(y_pred)
        except KeyboardInterrupt:
            break
        except:
            pred.append(0)
            traceback.print_exc()
    else: 
#         print('no model found')
        pred.append(0)
    if counter%1000 == 0:
        correlations.append(pearsonr(y_test[:len(pred)], pred)[0])
        print(f'Correlation: {pearsonr(y_test[:len(pred)], pred)[0]}, p-value = {pearsonr(y_test[:len(pred)], pred)[1]}, wallet = {wallet}')
    if counter%20000 == 0:
        for asset_id in range(0,13):
            try:
                if long[asset_id] ==1:
                    sell_price = dict_tmp[asset_id].tail(1).Close.values[0]
                    wallet += 100 * sell_price/buy_price[asset_id] - 100
            except: 
                continue
        print(f'Final wallet: {wallet}')
        break


In [ ]:
#check if things are working properly,the columns should be correct, and preds shouldn't be too small 
print(x_test)
pred[:10]

In [ ]:
# This assumes that we have minute-by-minute data for each asset

In [ ]:
## these numbers are all an artifact of that fact that predictions are all very close to 0.

# fixed lag, new XBGoost params:
    #params = {'lag':5,'less_features': False, 'shuffle':True} test_size = 10000, 0.0051152419695546245 (peak = 0.068)
    #params = {'lag':0,'less_features': True, 'shuffle':True, 'diff': False} test_size = 10000, 0.0051361107196150665 (peak = 0.06)
    #params = {'lag':0,'less_features': True, 'shuffle':True } test_size = 10000, 0.004907722808215038 (peak = 0.06)
    #params = {'lag':0,'less_features': True, 'shuffle':False } test_size = 10000, 0.004716585631229205 (peak = 0.06)

# previous code didn't run correctly, get_features(training) returns all nans except for the two shadow features.

# with scaling:
    # params = {'lag':10,'less_features': True, 'shuffle':False }, test_size = 10000, 0.03236088315425213
    
#with diff, no scaling & model = xgb.XGBRegressor(
#         n_estimators=500 (i think),
#         missing=-999,
#         random_state=1111,
#         tree_method='gpu_hist'  
#         ):
    # params = {'lag':5,'less_features': True, 'shuffle':False }, test_size = 10000, <0
    # params = {'lag':8,'less_features': True, 'shuffle':False }, test_size = 10000, <0
    # params = {'lag':10,'less_features': True, 'shuffle':False }, test_size = 10000, 0.059430976177124534
    # params = {'lag':10,'less_features': True, 'shuffle':True }, test_size = 10000, 0.007976245597496146
    # params = {'lag':20,'less_features': True, 'shuffle':True }, test_size = 10000, <0

    # with lagged features, lag = 3, test_size = 6000, shuffled, correlation for all assets: -0.025720808709807898

#with n_estimators = 1000:

    # params = {'lag':10,'less_features': True, 'shuffle':False }, test_size = 10000, 0.038023285612531356 (but high variance)

# all shit
    # with lagged features, lag = 3, test_size = 4000, correlation for all assets: -0.025720808709807898
    # with lagged features, lag = 3, test_size = 5000, correlation for all assets: 0.010013542572384845
    # with lagged features, lag = 3, test_size = 6000, correlation for all assets: 0.005208812545830533
    # with lagged features, lag = 1, test_size = 6000, correlation for asset_ID = 1: 0.012588759721880852
    # with lagged features,lag = 1, test_size = 6000, correlation for asset_ID = 0: -0.006493023259016858
    # with lagged features, lag = 3, test_size = 6000, correlation for asset_ID = 0: 0.025999920635118752
    # without lagged features, test_size = 6000, correlation for asset_ID = 1: <0
    # without lagged features, test_size = 6000, correlation for asset_ID = 0: <0

# Predict & submit

References: [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)

Something that helped me understand this iterator was adding a pdb checkpoint inside of the for loop:

```python
import pdb; pdb.set_trace()
```

See [Python Debugging With Pdb](https://realpython.com/python-debugging-pdb/) if you want to use it and you don't know how to.


In [ ]:
# env = gresearch_crypto.make_env()
# iter_test = env.iter_test()

# for i, (df_test, df_pred) in enumerate(iter_test):
#     display(df_test)
#     for j , row in df_test.iterrows():
        
#         if models[row['Asset_ID']] is not None:
#             try:
#                 model = models[row['Asset_ID']]
#                 x_test = get_features(row)
#                 y_pred = model.predict(pd.DataFrame([x_test]))[0]
#                 df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
#             except:
#                 df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
#                 traceback.print_exc()
#         else: 
#             df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
        
#     env.predict(df_pred)


